In [85]:
import pandas as pd
from helpers import sheets_api as sa
from helpers import constants as c

In [86]:
placements_by_game = sa.get_placements()
placements_by_game

,Timestamp,First Place,Second Place,Third Place,Fourth Place
0,12/16/2024 21:08:48,Thomas Pelowitz,Adin Tijerina,Clayton Eitel,Matt Schmall
1,12/16/2024 21:40:03,David Sabotta,James Berg,Thomas Pelowitz,Pablo


In [ ]:
player_cmd = pd.read_csv('data/player-cmd.csv')
player_cmd["Color Identity Textual"] = player_cmd["Color Identity"].apply(lambda x: c.COLOR_SYM_TO_NAME.get(x, x))
player_cmd

,Player,Commander,Color Identity,CMC,First Printing Set,Year of First Printing,Color Identity Textual
0,Adin Tijerina,Numot the Devastator,WUR,6,Planar Chaos,2007,Jeskai
1,Adrian Lagerwaard,"Radha, Heir to Keld",RG,2,Planar Chaos,2007,Gruul
2,Alex Nelson,"Teneb, the Harvester",WBG,6,Planar Chaos,2007,Abzan
3,Alexi Rotman,Sharuum the Hegemon,WUB,6,Shards of Alara,2008,Esper
4,Austin Jones,Thraximundar,UBR,7,Alara Reborn,2009,Grixis
5,Benny Garcia,"Crovax, Ascendant Hero",W,6,Time Spiral,2006,Mono-White
6,Blake Novy,"Lyzolda, the Blood Witch",BR,3,Dissension,2006,Rakdos
7,Branden Delany,Borborygmos,RG,7,Guildpact,2006,Gruul
8,Branden Lange,"Agrus Kos, Wojek Veteran",WR,5,Ravnica: City of Guilds,2005,Boros
9,Brian Fierro,Mayael the Anima,WRG,3,Shards of Alara,2008,Naya


In [88]:
player_stats = {}

for _, row in placements_by_game.iterrows():
    for column in ['First Place', 'Second Place', 'Third Place', 'Fourth Place']:
        player = row[column]
        if player not in player_stats:
            player_stats[player] = {'Total Points': 0, 'Games Played': 0}
        
        # Add the placement and increment the games played
        player_stats[player]['Total Points'] += c.PLACEMENT[column]
        player_stats[player]['Games Played'] += 1

placements_by_player = pd.DataFrame(data=player_stats).T
placements_by_player["Average Placement"] = placements_by_player["Total Points"] / placements_by_player["Games Played"]
placements_by_player

,Total Points,Games Played,Average Placement
Thomas Pelowitz,4,2,2.0
Adin Tijerina,2,1,2.0
Clayton Eitel,3,1,3.0
Matt Schmall,4,1,4.0
David Sabotta,1,1,1.0
James Berg,2,1,2.0
Pablo,4,1,4.0


In [89]:
def get_1v1_record(player1, player2, stringify = True):
    record = [0, 0]
    for index, row in placements_by_game.iterrows():
        if (player1 in row.values) and (player2 in row.values):
            col_1 = row[row == player1].index[0]
            col_2 = row[row == player2].index[0]

            if c.PLACEMENT[col_1] < c.PLACEMENT[col_2]:
                record[0] += 1
            else:
                record[1] += 1
    
    if not stringify:
        return record
    else:
        return f"{record[0]}:{record[1]}"

get_1v1_record('Thomas Pelowitz', 'Adin Tijerina')

'1:0'

In [90]:
def get_color_players(color: str, exact: bool) -> list:
    if exact:
        return player_cmd[player_cmd['Color Identity'] == color]["Player"].tolist()
    else:
        return player_cmd[player_cmd['Color Identity'].str.contains(color, case=False, na=False)]["Player"].tolist()

get_color_players("R", True)

['Clint Reiman', 'Tony Snyder']

In [ ]:
def get_player_record_against_color(player, color, exact, stringify):
    color_players = get_color_players(color, exact)

    overall_record = [0, 0]
    for player2 in color_players:
        v_player_record = get_1v1_record(player, player2, False)
        overall_record[0] += v_player_record[0]
        overall_record[1] += v_player_record[1]

    if not stringify:
        return overall_record
    else:
        return f"{overall_record[0]}:{overall_record[1]}"

get_player_record_against_color("Thomas Pelowitz", "W", False, True)

Adin Tijerina [1, 0]
Alex Nelson [0, 0]
Alexi Rotman [0, 0]
Benny Garcia [0, 0]
Branden Lange [0, 0]
Brian Fierro [0, 0]
David Sabotta [0, 1]
Garrett Strom [0, 0]
George Garrets [0, 0]
Jamie G [0, 0]
Josh Slawson [0, 0]
Justin Whiting [0, 0]
Mason Scheopper [0, 0]
Miles Larson [0, 0]
Nick Samson [0, 0]


'1:1'